# Query Notebook

In [ ]:
import json

In [ ]:
SAMPLE = True

In [ ]:
# This is how our queries will be formatted. They will later be created with
# user input
query = {
    "user": "4ZaqBJqt7laPPs8xfWvr6A",
    "types": ["restaurants"], # TODO I think this should be just one value out a list of options to make our work easier -Joshua
    "keywords": ["French", "cozy"],
    "literals": [("distance", "less than", 2), ("rating", "greater than", 3)],
    "k": 5
}

Program flow

1. Query the database for a list of locations that match the type of location
2. Narrow down the list to make sure they match the literals
3. Match keywords to the location
4. Apply the friend rating filter
5. Return K results